## Import packages

In [1]:
import sys
import time
import os
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer

#import matplotlib.pyplot as plt

## Set up for imports of .py modules by adding path to sys.path

In [2]:
path = Path(os.getcwd())
path = str(path)
print(path)
sys.path.insert(1, path)

/home/peter/Desktop/MLWorkflows/Regression


## Import python modules

In [3]:
import utils.general as general_utils

## Helpful functions

In [4]:
print("No functions")

No functions


## Parameters

In [5]:
# Data
data_file_path = 'data/source/Iris.csv'
target_variable = 'PetalWidthCm'

# Step 2
test_ratio = 0.20
split_random_state = 500

# Step 3
use_validation_set = True
validation_ratio = 0.30
validation_split_random_state = 1000

# Step 5
missingness_threshold = 0.20

# Step 7
ridge_random_state = 1500
lasso_random_state = 2000
elastic_net_random_state = 2500
target_encoder_random_state = 3000

## Set up to time script run time

In [6]:
start = time.time()

## Read in the data and get the size of the data.

In [7]:
df = pd.read_csv(data_file_path)
print(df.shape)
df.head()

(150, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


## 1. Check for missingness in target vector

In [8]:
print(df.shape)
df = df.dropna(subset=target_variable)
print(df.shape)

(150, 6)
(150, 6)


## 2. Train/Test Split

### Split the data

In [9]:
training_data, testing_data, training_target, testing_target = \
    general_utils.split_data(
        df,
        target_variable,
        test_ratio,
        split_random_state
    )

### Make sure every sample has a result

In [10]:
assert (list(training_data.index) == list(training_target.index))
assert (list(testing_data.index) == list(testing_target.index))

### Delete original dataframe

In [11]:
del df

### Save off the test data and delete the test dataframes to prevent leakage

In [12]:
pd.concat([testing_data, testing_target], axis=1).to_csv('data/test/Iris_test.csv', index=True, index_label='index')

In [13]:
del testing_data
del testing_target

## 3. Train/Validation split

In [14]:
if use_validation_set:
    training_data, validation_data, training_target, validation_target = \
    general_utils.split_data(
        pd.concat([training_data, training_target], axis=1),
        target_variable,
        test_ratio,
        split_random_state
    )

In [15]:
if use_validation_set:
    assert (list(training_data.index) == list(training_target.index))
    assert (list(validation_data.index) == list(validation_target.index))

In [16]:
if use_validation_set:
    pd.concat([validation_data, validation_target], axis=1).to_csv('data/validation/Iris_validation.csv', index=True, index_label='index')

In [17]:
if use_validation_set:
    del validation_data
    del validation_target

## 4. Identify attributes with  missingness above threshold

In [18]:
missingness_drop_list = general_utils.calculate_missingness(training_data, missingness_threshold)
missingness_drop_list

Id               0.0
SepalLengthCm    0.0
SepalWidthCm     0.0
PetalLengthCm    0.0
Species          0.0
dtype: float64


[]

## 5. Identify non machine learning attributes. Id columns, for example

In [19]:
general_utils.print_uniqueness(training_data)

The data frame has 96 rows

Id has 96 unique values and is dtype int64 examine more closely
SepalLengthCm has 34 unique values and is dtype float64 
SepalWidthCm has 20 unique values and is dtype float64 
PetalLengthCm has 43 unique values and is dtype float64 
Species has 3 unique values and is dtype object 


['Id']

In [20]:
non_machine_learning_attributes = ['Id'] # Fill this in manually with suspects of above.

## 6. Establish machine learning attribute configuration

In [21]:
ml_ignore_list = missingness_drop_list + non_machine_learning_attributes
ml_ignore_list

['Id']

In [22]:
# identify the remaining numerical attributes to be used in machine learning and enter them into the 
# numerical_attr list below.

numerical_attributes = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm']

# identify the remaining nominal attributes to be used in machine learning and enter them into the 
# nominal_attr list below.

nominal_attributes = ['Species']

print(f'ml_ignore_list: {ml_ignore_list}')
print(f'\nnumerical_attr: {numerical_attributes}')
print(f'nominal_attr: {nominal_attributes}')

print(f'\nnumber of machine learning attributes: {len(numerical_attributes) + len(nominal_attributes)}')
print(f'\nnumerical_attr and nominal_attr: {numerical_attributes + nominal_attributes}')

assert(training_data.shape[1] == len(ml_ignore_list) + len(nominal_attributes) + len(numerical_attributes))  # got them all?

ml_ignore_list: ['Id']

numerical_attr: ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm']
nominal_attr: ['Species']

number of machine learning attributes: 4

numerical_attr and nominal_attr: ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'Species']


## 7. Build a dictionary of default estimators

In [23]:
e = [
    ('LinearRegression', LinearRegression(
        fit_intercept=True,
        copy_X=True,
        n_jobs=None,
        positive=False
    )),
    ('Ridge', Ridge(
        alpha=1.0,
        fit_intercept=True,
        copy_X=True,
        max_iter=None,
        tol=0.0001,
        solver='auto',
        positive=False,
        random_state=2500
    )),
    ('Lasso', Lasso(
        alpha=1.0,
        fit_intercept=True,
        precompute=False,
        copy_X=True,
        max_iter=1000,
        tol=0.0001,
        warm_start=False,
        positive=False,
        random_state=2500,
        selection='cyclic'
    )),
    ('ElasticNet', ElasticNet(
        alpha=1.0, 
        l1_ratio=0.5, 
        fit_intercept=True, 
        precompute=False, 
        max_iter=1000, 
        copy_X=True, 
        tol=0.0001, 
        warm_start=False, 
        positive=False, 
        random_state=elastic_net_random_state, 
        selection='cyclic' 
    ))
]

estimators = {

    e[0][0]: e[0][1], # LinearRegression
    e[1][0]: e[1][1], # Ridge
    e[2][0]: e[2][1], # Lasso
    e[3][0]: e[3][1], # ElasticNet
}

### Build a preprocessor pipeline

In [24]:
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer()),
        ("scaler", StandardScaler())
    ]
)

In [25]:
nominal_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy='most_frequent')),
        ("target_encoder", TargetEncoder(
                    categories='auto', 
                    target_type='continuous', 
                    smooth='auto', 
                    cv=5, 
                    shuffle=True, 
                    random_state=target_encoder_random_state
                )
        ),
        ("scaler", StandardScaler())
    ]
)

In [26]:
preprocessor = ColumnTransformer(
        transformers=[
            ('nominal', nominal_transformer, nominal_attributes),
            ('numerical', numerical_transformer, numerical_attributes)
        ]
)

NameError: name 'ColumnTransformer' is not defined

## 10. survey (fit and evaluate)  default composite estimators

### since we are only working with one estimator our survey is over just one estimator

### fit the composite estimator

In [ ]:
composite_estimator.fit(train_cap_x_df, train_y_df.values.ravel())

### check out the regression coefficients and the intercept

In [ ]:
# what are the estimated coefficients for the linear regression problem

composite_estimator[1].coef_

In [ ]:
# what is the intercept of the linear regression problem

composite_estimator[1].intercept_

In [ ]:
# access the attribute names going into the estimator step of the pipeline

composite_estimator[0].get_feature_names_out()

In [ ]:
# make a dictionary that has the attribute names as keys and the corresponding estimated coefficients for the linear regression problem
# as values

names = ['intercept'] + [attr_name.split('_')[2] for attr_name in composite_estimator[0].get_feature_names_out()]
coef_values = [composite_estimator[1].intercept_] + list(composite_estimator[1].coef_)
lin_reg_parameters_dict = dict(zip(names, coef_values))

lin_reg_parameters_dict

### Note that these are equal to those found using the stasmodel OLS API. 

### evaluate the training error using root mean squared error (rmse)

In [ ]:
train_rmse = root_mean_squared_error(train_y_df, composite_estimator.predict(train_cap_x_df))
print(f'train_rmse: {train_rmse}')

print(f'relative train_rmse: {train_rmse/np.mean(train_y_df)}')

### evaluate the model using rmse and n-fold cross validation on the train set

The following code randomly splits the training set into 5 nonoverlapping subsets called folds, then it trains and evaluates the linear regression model 5 times, picking a different fold for evaluation every time and using the other 4 folds for training. The result is an array containing the 5 evaluation scores.

In [ ]:
cv_rmse = cross_val_score(
    estimator=composite_estimator, 
    X=train_cap_x_df,
    y=train_y_df.values.ravel(),
    groups=None, 
    scoring='neg_root_mean_squared_error', 
    cv=5, 
    n_jobs=None, 
    verbose=0, 
    fit_params=None, 
    params=None, 
    pre_dispatch='2*n_jobs', 
    error_score=np.nan
)
cv_rmse = -1 * cv_rmse
cv_rmse

In [ ]:
print(f'estimated model rmse: {np.mean(cv_rmse)}')
print(f'precision of estimated model rmse: {np.std(cv_rmse, ddof=1)}')

### plot predicted vs actual

In [ ]:
plot_pred_vs_actual(composite_estimator.predict(train_cap_x_df), train_y_df, 'train')

## 11. short list default composite estimators

### Note that since we are only working with one estimator there is no reason to short list

## 12. tune hyperparameters of short-listed composite estimators

### Note that the statsmodels OLS API does not have any model flexibility hyper parameters so we cannot tune the estimator.

## 13. evaluate tuned composite estimators

### Note that the statsmodels OLS API does not have any model flexibility hyper parameters so we cannot evaluate a tuned estimator.

## 14. check for false discoveries

### This is an advanced topic and will be covered latter in the course.

## 15. select a model

### Model selection is trivial as we only have one model in this modeling effort.

## 16. evaluate generalization -  make predictions on the test set with the model

### load the test set

In [ ]:
test_df = pd.read_csv('test_df.csv', index_col='index')
test_df.index.name = None
test_df.head()

### break test_df apart into the design matrix (test_cap_x_df) and the target vector (test_y_df) then delete test_df

In [ ]:
test_cap_x_df = test_df.iloc[:, :-1]
test_cap_x_df.head()

In [ ]:
test_y_df = test_df.iloc[:, -1].to_frame()
test_y_df.head()

In [ ]:
del test_df

### evaluate the model on the test data using the rmse metric

In [ ]:
test_rmse = root_mean_squared_error(test_y_df, composite_estimator.predict(test_cap_x_df))
print(f'test_rmse: {test_rmse}')

print(f'relative test_rmse: {test_rmse/np.mean(test_y_df)}')

### summary of results

| 5-fold cv on train (rmse) | test rmse |
|:-------------------------:|:---------:|
|       9.7 +/- 0.6         |    9.5    |

### Since the test set rmse of 9.5 falls within the train set 5-fold cv precision band (9.1 to 10.3) on the train set we can conclude that we are not overfitting the model. It is possible that a more flexible model might provide better predictions (lower rmse).

## check out script run time

In [ ]:
end = time.time()
print(f'script run time: {(end - start)/60} minutes')